In [528]:
import pandas as pd
df_train = pd.read_csv('clean_train.csv')
y = df_train['Survived']
df_train = df_train.drop(columns=['Survived'])
df_test = pd.read_csv('clean_test.csv')
df_train.head()

,Pclass,Age,SibSp,Parch,Fare,TicketDigits,nanAge,nanFare,nanEmbarked,nanCabinLetter,...,FirstName_Yousseff,FirstName_of,CabinLetter_A,CabinLetter_B,CabinLetter_C,CabinLetter_D,CabinLetter_E,CabinLetter_F,CabinLetter_G,CabinLetter_T
0,1.540963,4.003419,0.730463,0.0,2.482401,23.034467,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.730463,0.0,0.0
1,0.730463,4.882973,0.730463,0.0,6.002978,22.222542,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.730463,0.0,0.0,0.000000,0.0,0.0
2,1.540963,4.263161,0.000000,0.0,2.590967,56.087122,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.730463,0.0,0.0
3,0.730463,4.745132,0.730463,0.0,5.464102,31.557002,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.730463,0.0,0.0,0.000000,0.0,0.0
4,1.540963,4.745132,0.000000,0.0,2.610301,39.015194,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.730463,0.0,0.0


In [529]:
def print_moderate_correlations(df, col_to_correlate):
    cols = df[df.columns].corr().columns
    if df[col_to_correlate].dtype.name == 'category':
        df[col_to_correlate] = df[col_to_correlate].cat.codes
    corrs = df[df.columns].corr()[col_to_correlate]
    for col, corr in zip(cols, corrs):
        if abs(corr) >= 0.05 and col != col_to_correlate:
            print(col, ': ', corr)

In [530]:
df_train['Survived'] = y
print_moderate_correlations(df_train, 'Survived')
df_train = df_train.drop(columns=['Survived'])

Pclass :  -0.33556724781767644
Age :  -0.09299652651937937
Parch :  0.11116681373604048
Fare :  0.3293529559465449
TicketDigits :  -0.13683141810329266
familySize :  0.09566212847632743
isAlone :  -0.2033670856998909
is_french :  0.07364228638655174
is_religious :  -0.0649876556555749
is_nobility :  0.05056144542486103
Sex_female :  0.5433513806577505
Sex_male :  -0.5433513806577533
Embarked_C :  0.1682404312182353
Embarked_S :  -0.149682723270684
LastName_Asplund :  0.05056144542486187
LastName_Baclini :  0.08508270816321341
LastName_Beane :  0.06009484737835691
LastName_Becker :  0.060094847378357345
LastName_Beckwith :  0.06009484737835727
LastName_Bishop :  0.06009484737835722
LastName_Brown :  0.05056144542486092
LastName_Caldwell :  0.06009484737835746
LastName_Chambers :  0.060094847378356714
LastName_Coutts :  0.06009484737835714
LastName_Daly :  0.06009484737835697
LastName_Dick :  0.060094847378356755
LastName_Doling :  0.060094847378357436
LastName_Duff Gordon :  0.060094847

In [531]:
def remove_weak_correlations(df1: pd.DataFrame, df2, y, weak_threshold: float = 0.1) -> pd.DataFrame:
    """
    Removes weak correlations
    :param df: pandas DataFrame to remove columns from.
    :param col_to_correlate: String column name to check correlation with
    :param weak_threshold: float number that counts as an absolute weak threshold
    :return: pandas DataFrame without the columns weakly correlated to target
    """
    col_to_correlate = 'PREDICTOR_TO_CHECK_WEAK_CORRS'
    df1[col_to_correlate] = y
    cols = df1[df1.columns].corr().columns
    corrs = df1[df1.columns].corr()[col_to_correlate]
    weakly_correlated = list()
    for col, corr in zip(cols, corrs):
        if abs(corr) >= weak_threshold and col != col_to_correlate:
            weakly_correlated.append(col)
    print('Num of weakly correlated:', len(weakly_correlated))
    print('Total length:', len(df1.columns) - 1)
    df1 = df1.drop(columns=[col_to_correlate])
    return df1.drop(columns=weakly_correlated), df2.drop(columns=weakly_correlated)

In [532]:
def get_rid_feature(df, df2, feat):
    for column in df_train:
        if column.startswith(feat):
            df_train.drop(columns=[column], inplace=True)
            df_test.drop(columns=[column], inplace=True)
    return df_train, df_test

In [533]:
df_train, df_test = remove_weak_correlations(df_train, df_test, y)

Num of weakly correlated: 20
Total length: 1470


In [534]:
# df_train, df_test = get_rid_feature(df_train, df_test, 'Parch')
# df_train, df_test = get_rid_feature(df_train, df_test, 'LastName')
# df_train, df_test = get_rid_feature(df_train, df_test, 'FirstName')
# df_train, df_test = get_rid_feature(df_train, df_test, 'Sex_male')
# df_train, df_test = get_rid_feature(df_train, df_test, 'SibSp')
# df_train, df_test = get_rid_feature(df_train, df_test, 'Fare')
# df_train, df_test = get_rid_feature(df_train, df_test, 'Age')

In [535]:
df_train.head()

,Age,SibSp,nanAge,nanFare,nanEmbarked,nanCabinLetter,familySize,is_french,is_spanish,is_religious,...,FirstName_Woolf,FirstName_Wyckoff,FirstName_Ylio,FirstName_Yoto,FirstName_Youssef,FirstName_Yousseff,FirstName_of,CabinLetter_A,CabinLetter_G,CabinLetter_T
0,4.003419,0.730463,0.0,0.0,0.0,0.0,0.730463,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.882973,0.730463,0.0,0.0,0.0,0.0,0.730463,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.263161,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.745132,0.730463,0.0,0.0,0.0,0.0,0.730463,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.745132,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [536]:
def print_moderate_correlations(df, col_to_correlate):
    cols = df[df.columns].corr().columns
    if df[col_to_correlate].dtype.name == 'category':
        df[col_to_correlate] = df[col_to_correlate].cat.codes
    corrs = df[df.columns].corr()[col_to_correlate]
    for col, corr in zip(cols, corrs):
        if abs(corr) > 0.1 and col != col_to_correlate:
            print(col, ': ', corr)

In [537]:
# df_train, df_test = get_rid_feature(df_train, df_test, 'CabinLetter')

In [538]:
df_train.head()

,Age,SibSp,nanAge,nanFare,nanEmbarked,nanCabinLetter,familySize,is_french,is_spanish,is_religious,...,FirstName_Woolf,FirstName_Wyckoff,FirstName_Ylio,FirstName_Yoto,FirstName_Youssef,FirstName_Yousseff,FirstName_of,CabinLetter_A,CabinLetter_G,CabinLetter_T
0,4.003419,0.730463,0.0,0.0,0.0,0.0,0.730463,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.882973,0.730463,0.0,0.0,0.0,0.0,0.730463,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.263161,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.745132,0.730463,0.0,0.0,0.0,0.0,0.730463,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.745132,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [539]:
from sklearn.model_selection import train_test_split

x = df_train.values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [540]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [541]:
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(criterion='friedman_mse', init=None,
             learning_rate=0.05, max_depth=3,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=10,
             min_weight_fraction_leaf=0.0, n_estimators=12000,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

gb_clf.fit(x_train, y_train)
gb_clf.score(x_test, y_test)

0.6983240223463687

In [542]:
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier

gb2_clf = GradientBoostingClassifier(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   random_state = 5)

gb2_clf.fit(x_train, y_train)
gb2_clf.score(x_test, y_test)

0.6815642458100558

In [543]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(penalty='l2', tol=0.0001, solver='lbfgs',
                           C=1, fit_intercept=True, intercept_scaling=1.0, 
                           class_weight=None, random_state=None)
lr_clf.fit(x_train, y_train)
lr_clf.score(x_test, y_test)

0.659217877094972

In [544]:
df_sub = pd.read_csv('gender_submission.csv')
df_sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [545]:
sub_model = gb_clf

In [546]:
df_sub['Survived'] = gb_clf.predict(df_test.values)
df_sub.to_csv('submission.csv', index=False)